In [15]:
#imports 

import numpy as np
import os, tempfile
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import Sequential

from sklearn.dummy import DummyClassifier

import time
import scipy
from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
from tensorflow.keras.preprocessing import image_dataset_from_directory

import joblib
from keras.applications import VGG19
from keras import models
from keras import layers
from keras import optimizers



## Image Load In

In [2]:
# load in the train data

#starting from main project directory

train_normal_dir = "./data/chest_xray/train/NORMAL"
train_pneumonia_dir = "./data/chest_xray/train/PNEUMONIA"

imgs_train_normal = [file for file in os.listdir(train_normal_dir) if file.endswith('.jpeg')]
imgs_train_pneumonia = [file for file in os.listdir(train_pneumonia_dir) if file.endswith('.jpeg')]

In [3]:
test_normal_dir = "./data/chest_xray/test/NORMAL"
test_pneumonia_dir = "./data/chest_xray/test/PNEUMONIA"

imgs_test_normal = [file for file in os.listdir(test_normal_dir) if file.endswith('.jpeg')]
imgs_test_pneumonia = [file for file in os.listdir(test_pneumonia_dir) if file.endswith('.jpeg')]

In [4]:
num_train = len(imgs_train_normal) + len(imgs_train_pneumonia)
num_test = len(imgs_test_normal) + len(imgs_test_pneumonia)
num_train, num_test

(5216, 624)

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [6]:
#rescale images to 64 by 64
#create validation set as 20% of train set

train_folder = "./data/chest_xray/train"
train_gen = train_datagen.flow_from_directory(
        train_folder, 
        target_size=(64, 64), 
       
        class_mode='binary',
        subset='training',
        batch_size= num_train)

validation_gen = train_datagen.flow_from_directory(
        train_folder, 
        target_size=(64, 64), 
       
        class_mode='binary',
        subset='validation',
        batch_size= num_train)

test_folder = "./data/chest_xray/test"
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_folder, 
        target_size=(64, 64), 
        class_mode='binary', 
        batch_size= num_test)

Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [7]:
train_images, train_labels = next(train_gen)
val_images, val_labels = next(validation_gen)
test_images, test_labels = next(test_gen)

In [8]:
#(image height, image width, color channels)
train_images[0].shape

(64, 64, 3)

In [9]:
#preserving shape for VGG19


In [10]:
##adding early stop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping 

stop = EarlyStopping(monitor='loss', mode='min', patience=10)
callbacks = [stop]

## VGG19 with Transfer Learning/added layers

In [16]:
cnn_base = VGG19(weights='imagenet', 
                 include_top=False, 
                 input_shape=(64, 64, 3))


In [17]:
VGGtransfer_model = models.Sequential()
VGGtransfer_model.add(cnn_base)
VGGtransfer_model.add(layers.Flatten())
VGGtransfer_model.add(layers.Dense(128, activation='relu'))
VGGtransfer_model.add(layers.Dense(1, activation='sigmoid'))

In [18]:
cnn_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

In [19]:
VGGtransfer_model.compile(loss='binary_crossentropy',
                optimizer='RMSprop',
                metrics=['accuracy', tf.keras.metrics.Precision(),  tf.keras.metrics.Recall()])

In [ ]:
VGGtransfer_model.fit(train_images, train_labels, epochs=25, batch_size=256, callbacks = [stop], validation_data=(val_images, val_labels))

In [ ]:
##Played with multiple versions of VGG19 base and with transfer learning layers, the best result for VGG19=accuracy: 0.9286 - recall:0.9821